#### This Notebook is the sample Demo setup for working with DakkoAI Client Library


In [ ]:
# Import dakko library
from dakkoai import (
    DakkoAIClient,
    DakkoRegistrationModel,
    DakkoSearch,
)

# Import supporting DataScience lib
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
from collections import Counter
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import reduce
import pandas as pd

##### We will first create DakkoAI Client instance that we will be using throughout this Notebook


In [ ]:
# Create dakko_client
dakko_client = DakkoAIClient()

#### Next we have two options

1. We can register a new developer account with Dakko
2. We can also use pre-registered account

These accounts will be used to fetch OAuth Access_Token that is mandatory to access any API from Dakko AI.


In [ ]:
# Register new crypto Data Scientist
user_data = DakkoRegistrationModel(
    email="user@example.com", first_name="John", last_name="Doe", user_name="johndoe"
)

status = dakko_client.register(user_data)
print(f"Register User: {status}")
print(dakko_client.client_credentials.model_dump())

##### Now we will fetch the Access Token that will be used later


In [ ]:
# Get access credentials for further connection to Dakko
status = dakko_client.get_access_token()

print(f"Getting access token: {status}")
print(dakko_client.access_credentials.model_dump())

#### Search and fetch Pool information from DakkoAI

##### DakkoAI SDK provides two options for each execution

1. Fetch raw data as JSON
2. Fetch data in dataframe object that can be directly consumed for further analysis


In [ ]:
# Search for Significant events from Dakko
dakko_search_significant_events = DakkoSearch(
    query="liquidity_change", limit=10, offset=0
)

se_result = dakko_client.get_significant_events(dakko_search_significant_events)

print("Search for Top Significant Events Dataframe")
se_result.dataframe

#### Search Pool information


In [ ]:
# Search for Pools information
dakko_search_pool = DakkoSearch()

pools = dakko_client.get_pools(dakko_search_pool)

print("Search for Pools")
pools.dataframe

#### Run a SQL query against DakkoAI clean labelled data

These results will be utilized to do OHLCV analysis


In [ ]:
# Fire a query on clean data tables and get raw JSON data
SEARCH_ADDRESS = "0xAD9eF19e289dcbC9AB27b83D2dF53CDEFF60f02D".lower()

dakko_sql = DakkoSearch(
    query=f"""
    
    WITH 
        Constants AS (
            SELECT '{SEARCH_ADDRESS}' AS contract, 
            18 AS dec0, 
            18 AS dec1
        ), 
        PreparedData AS (
            SELECT 
                DATE_TRUNC('hour', timestamp(`timestamp`)) AS ts, 
                pow(sqrtPriceX96 / pow(2, 96), 2) * pow(10, dec0 - dec1) AS price, 
                abs(amount0) / pow(10, dec0) AS amount0, 
                liquidity / pow(10, 12) AS liquidity_e12 
            FROM dakko_poc.eth_demo.token_swaps_raw, Constants
            WHERE address = lower(contract)
        ) 
        
        SELECT 
            ts, 
            FIRST(price) AS open, 
            MAX(price) AS high, 
            MIN(price) AS low, 
            LAST(price) AS close, 
            SUM(amount0) AS raw_volume, 
            SUM(amount0 * price) AS volume, 
            SUM(liquidity_e12) AS liquidity_e12 
        FROM PreparedData 
        WHERE ts IS NOT NULL 
        GROUP BY ts 
        ORDER BY ts
    
    """
)

##### Plot the OHLCV sticks for the eth address


In [ ]:
# Plot the OHLCV sticks using Dataframe results from DakkoAI
temp_df = dakko_client.get_query_results(dakko_sql).dataframe

# let's draw the OHLCV analysis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Candlestick(
        x=temp_df.index,  # date values
        open=temp_df["open"],
        high=temp_df["high"],
        low=temp_df["low"],
        close=temp_df["close"],
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(
        x=temp_df.index,
        y=temp_df["volume"],
        marker={
            "color": "rgba(128,128,128,0.4)",
        },
    ),
    secondary_y=False,
)
fig.layout.yaxis2.showgrid = False

# change figsize
fig.update_layout(
    # autosize=False,
    # width=1000,
    height=800,
)

fig.show()

##### Plot the OHLCV for Price


In [ ]:
query_result = dakko_client.get_query_results(dakko_sql).dataframe

In [ ]:
query_result

In [ ]:
# Extracting OHLCV data
ohlc_data = dakko_client.get_query_results(dakko_sql).dataframe
ohlc_data["ts"] = pd.to_datetime(ohlc_data["ts"])

# Plotting the OHLCV chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plotting line connecting opens
ax.plot_date(ohlc_data["ts"], ohlc_data["open"], "k-")
# Plotting line connecting closes
ax.plot_date(ohlc_data["ts"], ohlc_data["close"], "k-")
ax.vlines(
    ohlc_data["ts"],
    ohlc_data["high"],
    ohlc_data["low"],
    colors="k",
    linestyles="solid",
)

# Formatting x-axis as dates
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d %H:%M:%S"))
plt.xticks(rotation=45)

# Setting labels and title
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("OHLCV Analysis")

plt.show()

#### Create Time series analysis on the closing price


In [ ]:
# Plotting the time series graph
plt.figure(figsize=(10, 6))
plt.plot(ohlc_data["ts"], ohlc_data["close"], marker="o", linestyle="-")
plt.title("Closing Prices Over Time")
plt.xlabel("Time")
plt.ylabel("Closing Price")
plt.grid(True)
plt.show()

#### Let's now plot sync event data


In [ ]:
SEARCH_ADDRESS = "0x704aD8d95C12D7FEA531738faA94402725acB035".lower()

dakko_sync_sql = DakkoSearch(
    query=f"""
    
    select * 
    from dakko_poc.eth_bronze.parsed_transaction_logs_refine 
    where address = '{SEARCH_ADDRESS}' 
    and event = 'Sync'
    
    """
)

sync_events = dakko_client.get_query_results(dakko_sync_sql).dataframe


def parse_args(df, names: str, args: str):
    # get first row of the column

    unique_names = df[names].unique()
    new_columns = [x.split(",") for x in unique_names]
    new_columns = reduce(lambda x, y: x + y, new_columns)

    df[new_columns] = None

    new_df = pd.DataFrame()

    for combo in unique_names:
        slice_df = df[df[names] == combo].copy()
        slice_df[combo.split(",")] = slice_df[args].str.split(";", expand=True)
        new_df = pd.concat([new_df, slice_df])

    new_df.drop(columns=[args, names], inplace=True)

    return new_df


sync_events["not_indexed_names"] = "reserve0,reserve1"
sync_events = parse_args(sync_events, "not_indexed_names", "not_indexed_args")
sync_events["timestamp"] = pd.to_datetime(sync_events["timestamp"])

dec0 = 1e18
dec1 = 1e18

sync_events["reserve0"] = sync_events["reserve0"].astype(float) / dec0
sync_events["reserve1"] = sync_events["reserve1"].astype(float) / dec1
sync_events.dropna(inplace=True)
sync_events.sort_values("timestamp", inplace=True)

sync_events["liquidity"] = (
    sync_events["reserve0"] * sync_events["reserve1"] * dec0 * dec1
) ** 0.5


sync_events["liquidity_diff"] = sync_events["liquidity"].diff()

fig, ax = plt.subplots(figsize=(12, 8))


temp_df = (
    sync_events[["timestamp", "liquidity"]]
    .set_index("timestamp")
    .resample("1H")
    .mean()
    .fillna(method="ffill")
)


temp_df.plot(y="liquidity", ax=ax)

ax.scatter(
    [sync_events.timestamp[sync_events.liquidity_diff.abs().idxmax()]],
    [sync_events.liquidity[sync_events.liquidity_diff.abs().idxmax()]],
    color="red",
    label="fluctuation",
)

ax.legend()

#### Liquidity distribution for uni3 pools


In [ ]:
SEARCH_ADDRESS = "0xAD9eF19e289dcbC9AB27b83D2dF53CDEFF60f02D".lower()

dakko_ld_sql = DakkoSearch(
    query=f"""
    with 
        txhashes as (
            select distinct transactionHash 
            from dakko_poc.eth_bronze.parsed_transaction_logs_refine 
            where address = "{SEARCH_ADDRESS}" 
            and event = "Mint"
        ), 
        tokenIds as (
            select indexed_args 
            from dakko_poc.eth_bronze.parsed_transaction_logs_refine 
            where event in ("IncreaseLiquidity", "DecreaseLiquidity") 
            and transactionHash in (select transactionHash from txhashes)
        ) 
        select distinct * from (
            select * 
            from dakko_poc.eth_bronze.parsed_transaction_logs_refine 
            where address = "{SEARCH_ADDRESS}" 
            and event in ("Mint", "Burn") 
            union all 
            select * from 
            dakko_poc.eth_bronze.parsed_transaction_logs_refine 
            where event in ("IncreaseLiquidity", "DecreaseLiquidity") 
            and indexed_args in (select indexed_args from tokenIds)
        )
        join (select indexed_names, not_indexed_names, signature from dakko_poc.eth_static.event_registry_with_meta)
        using (signature)
        """
)

ld_df = dakko_client.get_query_results(dakko_ld_sql).dataframe
ld_df

Processing liquidity events


In [ ]:
liqiq = parse_args(ld_df, "indexed_names", "indexed_args")
liqiq = parse_args(liqiq, "not_indexed_names", "not_indexed_args")
liqiq["timestamp"] = pd.to_datetime(liqiq["timestamp"])

# copy events with token id
liqiq_id = liqiq[liqiq.event.isin(["DecreaseLiquidity", "IncreaseLiquidity"])].copy()

# copy events without token id
liquiq_not_id = liqiq[
    ~liqiq.event.isin(["DecreaseLiquidity", "IncreaseLiquidity"])
].copy()

# build mapping between positions and tick ranges
tokenId_to_ticks = (
    (
        liqiq_id.drop(columns=["tickUpper", "tickLower"]).merge(
            liquiq_not_id[["transactionHash", "tickUpper", "tickLower"]],
            how="inner",
            on="transactionHash",
        )
    )[["tokenId", "tickUpper", "tickLower"]]
    .drop_duplicates()
    .reset_index(drop=True)
    .to_dict("records")
)

# remap
tokenId_to_ticks_new = dict()

for x in tokenId_to_ticks:
    tokenId_to_ticks_new[x["tokenId"]] = (x["tickUpper"], x["tickLower"])

# apply for original df

liqiq["tickUpper"] = liqiq.apply(
    lambda x: (
        tokenId_to_ticks_new[x["tokenId"]][0]
        if x["tokenId"] is not None
        else x["tickUpper"]
    ),
    axis=1,
)
liqiq["tickLower"] = liqiq.apply(
    lambda x: (
        tokenId_to_ticks_new[x["tokenId"]][1]
        if x["tokenId"] is not None
        else x["tickLower"]
    ),
    axis=1,
)

# rebuild liquidity column using all events now
liqiq.liquidity = liqiq.apply(
    lambda x: float(x.liquidity) if x.liquidity is not None else float(x.amount), axis=1
)

# convert to numeric
for col in ["amount0", "amount1"]:
    liqiq[col] = liqiq[col].astype(float)

liqiq.tickLower = liqiq.tickLower.astype(int)
liqiq.tickUpper = liqiq.tickUpper.astype(int)

# add multiplier for corresponding events
for col in ["liquidity", "amount0", "amount1"]:
    liqiq[col] *= liqiq.event.apply(
        lambda x: -1 if x in ["DecreaseLiquidity", "Burn"] else 1
    )

In [ ]:
# extract liquidity distribution using liquidity dataframe preprocessed
def get_liq_ticks(df: pd.DataFrame, dec0: int = 18, dec1: int = 18):
    unique_ticks = sorted(set(df["tickLower"]).union(set(df["tickUpper"])))

    tick_liquidity = pd.DataFrame(unique_ticks, columns=["tick"]).set_index("tick")
    tick_liquidity["totalLiquidity"] = 0
    tick_liquidity["amount0locked"] = 0
    tick_liquidity["amount1locked"] = 0

    for _, row in df.iterrows():
        tick_liquidity.loc[
            row["tickLower"] : row["tickUpper"], "totalLiquidity"
        ] += row["liquidity"]
        tick_liquidity.loc[row["tickLower"] : row["tickUpper"], "amount0locked"] += row[
            "amount0"
        ]
        tick_liquidity.loc[row["tickLower"] : row["tickUpper"], "amount1locked"] += row[
            "amount1"
        ]

    tick_liquidity = tick_liquidity.reset_index()

    tick_liquidity["price"] = 1.0001 ** tick_liquidity["tick"] * 10 ** (dec1 - dec0)

    tick_liquidity["tick_width"] = tick_liquidity["tick"].diff().shift(-1).fillna(0)
    tick_liquidity["price_width"] = tick_liquidity["price"].diff().shift(-1).fillna(0)

    return tick_liquidity

In [ ]:
from datetime import timedelta

# last available liquidity state
tick_liquidity = get_liq_ticks(liqiq)

# liquidity state 4 days before last state available
tick_liquidity_past_snap = get_liq_ticks(
    liqiq[liqiq["timestamp"] < liqiq["timestamp"].max() - timedelta(days=4)]
)

Get two nearest swap events and use price from there


In [ ]:
max_ts = int(liqiq["timestamp"].max().timestamp())
ts_ago = int((liqiq["timestamp"].max() - timedelta(days=4)).timestamp())

SEARCH_ADDRESS = "0xAD9eF19e289dcbC9AB27b83D2dF53CDEFF60f02D".lower()

dakko_sql = DakkoSearch(
    query=f"""
    
    WITH 
        Constants AS (
            SELECT 
                '{SEARCH_ADDRESS}' AS contract, 
                18 AS dec0, 
                18 AS dec1,
                timestamp({max_ts}) AS max_ts,
                timestamp({ts_ago}) AS ts_ago,
                30 as search_width
        ), 
        PreparedData AS (
            SELECT 
                `timestamp`,
                pow(sqrtPriceX96 / pow(2, 96), 2) * pow(10, dec0 - dec1) AS price,
                tick 
            FROM dakko_poc.eth_demo.token_swaps_raw, Constants
            WHERE address = lower(contract)
        ) 
        
    SELECT * FROM (SELECT price, tick
    FROM PreparedData, Constants 
    WHERE abs(TIMESTAMPDIFF(SECOND, `timestamp`, max_ts)) < search_width
    ORDER BY `timestamp` DESC
    LIMIT 1)
    UNION ALL
    SELECT * FROM (SELECT price, tick
    FROM PreparedData, Constants 
    WHERE abs(TIMESTAMPDIFF(SECOND, `timestamp`, ts_ago)) < search_width
    ORDER BY `timestamp` DESC
    LIMIT 1)
    
    """
)

last_swaps = dakko_client.get_query_results(dakko_sql).dataframe
last_price, last_price_past_snap = last_swaps.price.values
last_tick, last_tick_past_snap = last_swaps.tick.astype(int).values

Plot liquidity distribution in tick space


In [ ]:
import matplotlib.pyplot as plt

fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 6))

ax.bar(
    tick_liquidity["tick"],
    tick_liquidity["totalLiquidity"],
    width=tick_liquidity["tick_width"],
    align="edge",
    alpha=0.6,
    color="violet",
    label="Liquidity",
)

ax2.bar(
    tick_liquidity["tick"],
    tick_liquidity["amount0locked"],
    width=tick_liquidity["tick_width"],
    align="edge",
    alpha=0.6,
    color="blue",
    label="amount0locked",
)

ax3.bar(
    tick_liquidity["tick"],
    tick_liquidity["amount1locked"],
    width=tick_liquidity["tick_width"],
    align="edge",
    alpha=0.6,
    color="red",
    label="amount1locked",
)

for a, col in zip((ax, ax2, ax3), ("totalLiquidity", "amount0locked", "amount1locked")):
    a.vlines(
        last_tick,
        0,
        tick_liquidity[col].max(),
        color="red",
        alpha=1,
        label="Current price",
    )

ax.set_ylabel("Liquidity", labelpad=15)
ax2.set_ylabel("SAVM locked", labelpad=15)
ax3.set_ylabel("WETH locked", labelpad=15)
ax.set_xlabel("Tick", labelpad=15)
ax2.set_xlabel("Tick", labelpad=15)
ax3.set_xlabel("Tick", labelpad=15)

ax.set_xlim(-8e4, -4e4)
ax2.set_xlim(-8e4, -4e4)
ax3.set_xlim(-8e4, -4e4)

ax.legend()
ax2.legend()
ax3.legend()

Liquidity distribution in price space


In [ ]:
fig, (ax, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 6))

ax.bar(
    tick_liquidity["price"],
    tick_liquidity["totalLiquidity"],
    width=tick_liquidity["price_width"],
    align="edge",
    alpha=0.6,
    color="violet",
    label="Liquidity",
)

ax2.bar(
    tick_liquidity["price"],
    tick_liquidity["amount0locked"],
    width=tick_liquidity["price_width"],
    align="edge",
    alpha=0.6,
    color="blue",
    label="amount0locked",
)

ax3.bar(
    tick_liquidity["price"],
    tick_liquidity["amount1locked"],
    width=tick_liquidity["price_width"],
    align="edge",
    alpha=0.6,
    color="red",
    label="amount1locked",
)

for a, col in zip((ax, ax2, ax3), ("totalLiquidity", "amount0locked", "amount1locked")):
    a.vlines(
        last_price,
        0,
        tick_liquidity[col].max(),
        color="red",
        alpha=1,
        label="Current price",
    )

ax.set_ylabel("Liquidity", labelpad=15)
ax2.set_ylabel("SAVM locked", labelpad=15)
ax3.set_ylabel("WETH locked", labelpad=15)
ax.set_xlabel("Price", labelpad=15)
ax2.set_xlabel("Price", labelpad=15)
ax3.set_xlabel("Price", labelpad=15)

ax.set_xlim(0, 1e-2)
ax2.set_xlim(0, 1e-2)
ax3.set_xlim(0, 1e-2)

ax.legend()
ax2.legend()
ax3.legend()

Plot liquidity distribution difference in tick space


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(
    tick_liquidity_past_snap["tick"],
    tick_liquidity_past_snap["totalLiquidity"],
    width=tick_liquidity_past_snap["tick_width"],
    align="edge",
    alpha=0.5,
    color="gray",
    label="Liquidity 4 days ago",
)
ax.vlines(
    last_tick_past_snap,
    0,
    tick_liquidity_past_snap["totalLiquidity"].max(),
    color="gray",
    linestyle="--",
    alpha=1,
    label="Price 4 days ago",
)

ax.bar(
    tick_liquidity["tick"],
    tick_liquidity["totalLiquidity"],
    width=tick_liquidity["tick_width"],
    align="edge",
    alpha=1,
    color="violet",
    label="Liquidity now",
)
ax.vlines(
    last_tick,
    0,
    tick_liquidity["totalLiquidity"].max(),
    color="red",
    alpha=1,
    label="Current price",
)

ax.set_xlim(-8e4, -4e4)
ax.legend()

ax.set_ylabel("Liquidity", labelpad=15)
ax.set_xlabel("Tick", labelpad=15)

Plot liquidity difference in price space


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(
    tick_liquidity_past_snap["price"],
    tick_liquidity_past_snap["totalLiquidity"],
    width=tick_liquidity_past_snap["price_width"],
    align="edge",
    alpha=0.5,
    color="gray",
    label="Liquidity 4 days ago",
)
ax.vlines(
    last_price_past_snap,
    0,
    tick_liquidity_past_snap["totalLiquidity"].max(),
    color="gray",
    linestyle="--",
    alpha=1,
    label="Price 4 days ago",
)

ax.bar(
    tick_liquidity["price"],
    tick_liquidity["totalLiquidity"],
    width=tick_liquidity["price_width"],
    align="edge",
    alpha=1,
    color="violet",
    label="Liquidity now",
)
ax.vlines(
    last_price,
    0,
    tick_liquidity["totalLiquidity"].max(),
    color="red",
    alpha=1,
    label="Current price",
)

ax.set_xlim(0, 1e-2)
ax.legend()

ax.set_ylabel("Liquidity", labelpad=15)
ax.set_xlabel("Price", labelpad=15)

#### Fetch Wallet Labels for Top wallets


In [ ]:
# Search for Wallet labels from Dakko
# dakko_search_wallets_labels = DakkoSearch(
#     query='kucoin'
# )

dakko_search_wallets_labels = DakkoSearch(query="binance")

# dakko_search_wallets_labels = DakkoSearch(
#     query='bybit'
# )

wallet_labels = dakko_client.get_wallet_labels(dakko_search_wallets_labels).dataframe

print("Search for Top Wallet Labels")
print(wallet_labels)

In [ ]:
# Search for Wallet labels from Dakko Dataframe
dakko_search_wallets_labels = DakkoSearch()

wallet_labels_dataframe = dakko_client.get_wallet_labels(
    dakko_search_wallets_labels
).dataframe

print("Search for Top Wallet Labels Dataframe")
print(wallet_labels_dataframe)